# Boat Sales Analysis

### This script contains the following:

#### 1. Importing Visualization Libraries and Data
#### 2. Data Wrangling
#### 3. Data Cleaning
#### 4. Exploring Relationships


## 1. Importing Libraries and Data

In [1]:
import quandl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import os
import warnings 

warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight') 

In [2]:
# Ensure charts are displayed in notebook
%matplotlib inline

In [3]:
# Path to the original data
path= r'/Users/mentaykoshzhanova/Desktop/boat project'

In [4]:
# Using path to create df 
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'boat_data.csv'), index_col = False)

## 2. Data Wrangling

In [5]:
df.shape

(9888, 10)

In [6]:
df.head()

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days
0,CHF 3337,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,NaN,Switzerland Â» Lake Geneva Â» VÃ©senaz,226
1,EUR 3490,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,Germany Â» BÃ¶nningstedt,75
2,CHF 3770,Sport Boat,Marine power boats,new boat from stock,0,3.69,1.42,Aluminium,Switzerland Â» Lake of Zurich Â» StÃ¤fa ZH,124
3,DKK 25900,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,NaN,Denmark Â» Svendborg,64
4,EUR 3399,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,Germany Â» Bayern Â» MÃ¼nchen,58


In [7]:
df.describe()

,Year Built,Length,Width,Number of views last 7 days
count,9888.000000,9879.000000,9832.000000,9888.000000
mean,1893.192860,11.570017,3.520124,149.160801
std,460.201582,6.002820,1.220534,151.819752
min,0.000000,1.040000,0.010000,13.000000
25%,1996.000000,7.470000,2.540000,70.000000
50%,2007.000000,10.280000,3.330000,108.000000
75%,2017.000000,13.930000,4.250000,172.000000
max,2021.000000,100.000000,25.160000,3263.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9888 entries, 0 to 9887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        9888 non-null   object 
 1   Boat Type                    9888 non-null   object 
 2   Manufacturer                 8550 non-null   object 
 3   Type                         9882 non-null   object 
 4   Year Built                   9888 non-null   int64  
 5   Length                       9879 non-null   float64
 6   Width                        9832 non-null   float64
 7   Material                     8139 non-null   object 
 8   Location                     9852 non-null   object 
 9   Number of views last 7 days  9888 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 772.6+ KB


#### There are null data and non suitable data in location column

## 3. Exploratory and Data Cleaning

In [9]:
# looking closer at price column
df['Price'].describe()

count          9888
unique         3182
top       EUR 65000
freq             77
Name: Price, dtype: object

In [10]:
# splitting currency from price and creating separate column for currency
df[['Currency', 'Price']]= df['Price'].str.extract('(\w+)\s(\w+)', expand=True)
df.head()

,Price,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Location,Number of views last 7 days,Currency
0,3337,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,NaN,Switzerland Â» Lake Geneva Â» VÃ©senaz,226,CHF
1,3490,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,Germany Â» BÃ¶nningstedt,75,EUR
2,3770,Sport Boat,Marine power boats,new boat from stock,0,3.69,1.42,Aluminium,Switzerland Â» Lake of Zurich Â» StÃ¤fa ZH,124,CHF
3,25900,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,NaN,Denmark Â» Svendborg,64,DKK
4,3399,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,Germany Â» Bayern Â» MÃ¼nchen,58,EUR


In [11]:
# creating a new column 'Price in usd' with converted prices from initial currency
def price_in_dollars(Price, Currency):
    if Currency == 'EUR':
        return int (Price) * 1.07
    elif Currency == 'CHF':
        return int(Price) * 1.11
    elif Currency == 'DKK':
        return int(Price) * 0.14
    elif Currency == 'Â£':
        return int(Price) * 1.24
df['Price in usd'] = df.apply(lambda x: price_in_dollars(x.Price, x.Currency), axis=1)

In [12]:
# looking closer at location column
df['Location'].describe()

count                               9852
unique                              2995
top       Netherlands Â» In verkoophaven
freq                                 309
Name: Location, dtype: object

In [13]:
# splitting location into 'Country' and 'City' columns
df[['Country', 'City']]= df['Location'].str.extract('(\w+)\s(\w+)', expand=True)


In [15]:
#dropping unnecessary columns
df_2 = df.drop(['Price','Location','Currency','City'], axis=1)
df_2.head()

,Boat Type,Manufacturer,Type,Year Built,Length,Width,Material,Number of views last 7 days,Price in usd,Country
0,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,NaN,226,3704.07,Switzerland
1,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,75,3734.30,Germany
2,Sport Boat,Marine power boats,new boat from stock,0,3.69,1.42,Aluminium,124,4184.70,Switzerland
3,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,NaN,64,3626.00,Denmark
4,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,58,3636.93,Germany


In [16]:
# checking for duplicates // no duplicates
df_2_dups= df[df.duplicated()]
df_2_dups.shape

(0, 14)

In [17]:
#Checking for missing values
df_2.isnull().sum()

Boat Type                         0
Manufacturer                   1338
Type                              6
Year Built                        0
Length                            9
Width                            56
Material                       1749
Number of views last 7 days       0
Price in usd                    298
Country                        1285
dtype: int64

In [18]:
# Renaming 'Type' to 'Condition' for readability
df_2.rename(columns = {'Type':'Condition'}, inplace = True) 

In [19]:
# fill Null in 'Type' column
df_2['Condition'].fillna(df_2['Condition'].value_counts().idxmax(),inplace=True)

In [20]:
# fill Null in 'Material'
df_2['Material'].fillna(df_2['Material'].value_counts().idxmax(),inplace=True)

In [21]:
# Impute missing values with median for price column
df_2['Price in usd'].fillna(df_2['Price in usd'].median(), inplace=True)
df_2['Length'].fillna(df_2['Length'].median(), inplace=True)
df_2['Width'].fillna(df_2['Width'].median(), inplace=True)

In [22]:
#Checking for missing values after cleaning
df_2.isna().sum()

Boat Type                         0
Manufacturer                   1338
Condition                         0
Year Built                        0
Length                            0
Width                             0
Material                          0
Number of views last 7 days       0
Price in usd                      0
Country                        1285
dtype: int64

In [23]:
df_2.shape

(9888, 10)

In [24]:
df_clean = df_2.dropna(subset=['Manufacturer','Country'])

In [25]:
df_2['Manufacturer'].fillna('None',inplace=True)
df_2['Country'].fillna('None',inplace=True)
df_2.isna().sum()

Boat Type                      0
Manufacturer                   0
Condition                      0
Year Built                     0
Length                         0
Width                          0
Material                       0
Number of views last 7 days    0
Price in usd                   0
Country                        0
dtype: int64

In [26]:
print('Before Clean :' ,df_2.shape)
print('After Clean :' ,df_clean.shape)

Before Clean : (9888, 10)
After Clean : (7420, 10)


In [27]:
df_clean.dtypes

Boat Type                       object
Manufacturer                    object
Condition                       object
Year Built                       int64
Length                         float64
Width                          float64
Material                        object
Number of views last 7 days      int64
Price in usd                   float64
Country                         object
dtype: object

In [28]:
df_clean.head()

,Boat Type,Manufacturer,Condition,Year Built,Length,Width,Material,Number of views last 7 days,Price in usd,Country
0,Motor Yacht,Rigiflex power boats,new boat from stock,2017,4.00,1.90,GRP,226,3704.07,Switzerland
1,Center console boat,Terhi power boats,new boat from stock,2020,4.00,1.50,Thermoplastic,75,3734.30,Germany
2,Sport Boat,Marine power boats,new boat from stock,0,3.69,1.42,Aluminium,124,4184.70,Switzerland
3,Sport Boat,Pioner power boats,new boat from stock,2020,3.00,1.00,GRP,64,3626.00,Denmark
4,Fishing Boat,Linder power boats,new boat from stock,2019,3.55,1.46,Aluminium,58,3636.93,Germany


In [29]:
#Exporting to csv
df_clean.to_csv(os.path.join(path, '02 Data','Prepared Data', 'clean_boat.csv'))